# LISTA 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import skew, binom
from statsmodels.graphics.mosaicplot import mosaic
import plot_likert

## CZĘŚĆ I

### ZADANIE 1

#### ZADANIE 1.1
    Wczytaj dane i przygotuj je do analizy. Zadbaj o odpowiednie typy zmiennych, zweryfikuj czy przyjmują wartości zgodne z powyższym opisem, zbadaj czy nie występują braki w danych.

#### ZADANIE 1.2
    Utwórz zmienną WIEK_KAT przeprowadzając kategoryzację zmiennej WIEK korzystając z następujących przedziałów: do 35 lat, mięedzy 36 a 45 lat, między 46 a 55 lat, powyżej 55 lat.


#### ZADANIE 1.3
    Sporządź tablice liczności dla zmiennych: DZIAŁ, STAŻ, CZY_KIER, PŁEĆ, WIEK_KAT. Sformułuj wnioski.

#### ZADANIE 1.4
     Sporządź wykresy kołowe oraz wykresy słupkowe dla zmiennych: PYT_1 oraz PYT_2. Sformułuj wnioski.

#### ZADANIE 1.5
    Sporządź tablice wielodzielcze dla par zmiennych: PYT_1 i DZIAŁ, PYT_1 i STAŻ, PYT_1 i CZY_KIER, PYT_1 i PŁEĆ oraz PYT_1 i WIEK_KAT. Sformułuj wnioski.

#### ZADANIE 1.6
    Sporządź tablicę wielodzielczą dla pary zmiennych: PYT_2 i PYT_3. Sformułuj wnioski.

#### ZADANIE 1.7
     Utwórz zmienną CZY_ZADOW na podstawie zmiennej PYT_2 łącząc kategorie "nie zgadzam się" i "zdecydowanie się nie zgadzam" oraz "zgadzam się" i "zdecydowanie się zgadzam".

#### ZADANIE 1.8
    Sporządź wykresy mozaikowe odpowiadające parom zmiennych: CZY_ZADOW i DZIAŁ, CZY_ZADOW i STAŻ, CZY_ZADOW i CZY_KIER, CZY_ZADOW i PŁEĆ oraz CZY_ZADOW i WIEK_KAT. Czy na podstawie uzyskanych wykresów można postawić pewne hipotezy dotyczące realicji między powyższymi zmiennymi? Spróbuj sformułować kilka takich hipotez.

## CZĘŚĆ II

### ZADANIE 2 
    Zilustruj odpowiedzi na pytanie "Jak bardzo zgadzasz się ze stwierdzeniem, że firma pozwala na (...)?" (zmienna PYT_1) w całej badanej grupie oraz w podgrupach ze względu na zmienną CZY_KIER. W tym celu możesz zaproponować własne metody wizualizacji lub zapozać się z biblioteką likert i dostępnymi tam funkcjami summary oraz plot (jeśli korzystarz z R) oraz z biblioteką Altair lub plot-likert (jeśli korzystarz z Pythona).

### ZADANIE 3
    Zapoznaj się z funkcją sample z biblioteki stats (w R) lub z funkcją random.choice z biblioteki numpy (w Pythonie). Przetestuj jej działanie dla różnych wartości argumentów wejściowych. Następnie wylosuj próbkę o liczności 10% wszystkich rekordów z pliku "ankieta.csv" w dwóch wersjach: ze zwracaniem oraz bez zwracania.

### ZADANIE 4
    Zaproponuj metodę symulowania zmiennych losowych z rozkładu dwumianowego. Napisz funkcję do generowania realizacji, a następnie zaprezentuj jej działanie porównując wybrane teoretyczne i empiryczne charakterystyki dla przykładowych wartości paramertów rozkładu: n i p.

### ZADANIE 5
    Zaproponuj metodę symulowania wektorów losowych z rozkładu wielomianowego. Napisz funkcję do generowania realizacji, a następnie zaprezentuj jej działanie porównując wybrane teoretyczne i empiryczne charakterystyki dla przykładowych wartości paramertów rozkładu: n i p.


## CZĘŚĆ III i IV

### ZADANIE 6
    Napisz funkcję do wyznaczania realizacji przedziału ufności Cloppera-Pearsona. Niech argumentem wejściowym będzie poziom ufności, liczba sukcesów i liczba prób lub poziom ufności i wektor danych (funkcja powinna obsługiwać oba przypadki).

### ZADANIE 7
    Korzystając z funkcji napisanej w zadaniu 6. wyznacz realizacje przedziałów ufności dla prawdopodobieństwa, że pracownik uważa szkolenia za przystosowane do swoich potrzeb w pierwszym badanym okresie oraz w drugim badanym okresie. Skorzystaj ze zmiennych CZY_ZADW oraz CZY_ZADW_2 (utwórz zmienną analogicznie jak w zadaniu 1.7). Przyjmij 1−α = 0.95.

### ZADANIE 8
     Zapoznaj się z funkcjami do generowania zmiennych losowych z rozkładu dwumianowego oraz do wyznaczania przedziałów ufności dla parametru p. Przetestuj ich działanie.

### ZADANIE 9
    Przeprowadź symulacje, których celem jest porównanie prawdopodobieństwa pokrycia i długości przedziałów ufności Cloppera-Pearsona, Walda i trzeciego dowolnego typu zaimplementowanego w wybranej funkcji. Rozważ 1−α = 0.95, rozmiar próby n ∈ {30,100,1000} i różne wartości prawdopodobieństwa p. Wyniki umieść na wykresach i sformułuj wnioski, które dla konkretnych danych ułatwią wybór konkretenego typu przedziału ufności.

     * W zadaniu 8. w pakiecie R możesz skorzystać z funkcji rbinom z biblioteki stats oraz binom.confint z biblioteki binom. W Pythonie możesz skorzystać z binomial z biblioteki random oraz stats.proportion.proportion_confint z biblioteki statsmodels.

## CZĘŚĆ V

### ZADANIE 10
    Zapoznaj siĘ z funkcjami służącymi do wykonania testu dokładnego oraz asympotycznego weryfikującego hipoteze zerową dotycząca prawodopodobieństwa sukcesu z rozkładu dwumianowego. W pakiecie R możesz skorzystać z binom.test oraz prop.test z biblioteki stats, nastomiast w Pythonie użyj stats.binomtest z biblioteki scipy oraz stats.proportion.proportions_ztest z biblioteki statsmodels. Przetestuj działanie funkcji.
    

### ZADANIE 11
    Dla danych z pliku "ankieta.csv" korzystajćc z funkcji z zadania 10., przyjmując 1−α =0.95, zweryfikuj następujące hipotezy i sformułuj wnioski:
     1. Prawdopodobieństwo, że w firmie pracuje kobieta wynosi 0.5.
     2. Prawdopodbieństwo, że pracownik uważa szkolenia za przystosowane do swoich potrzeb wpierwszym badanym okresie jest większe bądź równe 0.7.
     3. Prawdopodobieństwo, że kobieta pracuje na stanowisku kierowniczym jest równe prawdopodobieństwu, że mężczczyzna pracuje na stanowisku kierowniczym.
     4. Prawdopodobieństwo, że kobieta uważa szkolenia za przystosowane do swoich potrzeb w pierwszym badanym okresie jest równe prawdopodobieństwu, że mężczczyzna uważa szkolenia za przystosowane do swoich potrzeb w pierwszym badanym okresie.
     5. Prawdopodobieństwo, że kobieta pracuje w dziale zasobów ludzkich jest większe lub równe prawdopodobieństwu, że mężczczyzna pracuje w dziale zasobów ludzkich.

### ZADANIE 12
    Wyznacz symulacyjnie moc testu dokładnego oraz moc testu asymptotycznego w przypadku weryfikacji hipotezy zerowej H0 : p = 0.9 przeciwko H1 : p= 0.9 przyjmując wartość 1−α =0.95. Uwzględnij różne wartości alternatyw i różne rozmiary próby. Sformułuj wnioski.

----------------------------------------------------------------------

## Zadania dodatkowe

### ZADANIE *1. 
    Wyznacz granice asymptotycznego przedziału ufności dla prawdopodobieństwa sukcesu bazując na przekształceniu logit korzystając z metody delta. Zaimplementuj metodę oraz porównaj wyniki z funkcją zaimplementowaną w wybranym pakiecie.